In [7]:
import sqlite3
from sqlite3 import Error
 
 
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()
 
if __name__ == '__main__':
    create_connection(r"C:\Users\jpeng\OneDrive - Navigant Consulting Inc\Desktop\nyc rolling sales\nyc rolling sales\NYC_RealEstate_Sales_2003_2018.sqlite")

 

2.6.0


In [59]:
conn=sqlite3.connect(r'C:\Users\jpeng\OneDrive - Navigant Consulting Inc\Desktop\nyc rolling sales\nyc rolling sales\NYC_RealEstate_Sales_2003_2018.sqlite')
db_list = []
mycursor = conn.cursor()
for db_name in mycursor.execute("SELECT name FROM sqlite_master WHERE type = 'table'"):
    db_list.append(db_name)


In [64]:
tbl_list=[]
agg_list=[]
sale_count_list=[]
exp_list=[]
for i in range(2003,2019):
    tbl='yr_'+str(i)
    agg='agg_' + str(i)
    sc='sale_count'+str(i)
    temp='export'+str(i)
    tbl_list.append(tbl)
    agg_list.append(agg)
    sale_count_list.append(sc)
    exp_list.append(temp)
    

In [50]:
for j in range(0,len(tbl_list)):
    sql_text='create table ' + agg_list[j]+  ' as select borough as borough_id, name, nbhd, avg(price) as average_sale_price from ' + tbl_list[j] + ' a join borough b on' + ' a.borough=b.id \
where usable=="Y" \
group by borough, nbhd \
order by average_sale_price desc'
    crsr=mycursor.execute(sql_text)

In [55]:
for k in range(0,len(tbl_list)):
    sql_text2= 'create table ' + sale_count_list[k]+' as \
select nbhd, borough, count(distinct sale_id) as transaction_volume, count(distinct bbl_id) as property_count from '+tbl_list[k]+\
' where usable=="Y" \
group by borough, nbhd \
order by count(distinct sale_id) desc, count(distinct bbl_id) desc'

    mycursor.execute(sql_text2)

In [65]:
for l in range(0,len(tbl_list)):
    sql_text3= 'create table '+ exp_list[l]+' as \
select borough_id, b.nbhd, name, transaction_volume, property_count, average_sale_price \
from '+agg_list[l]+' a \
  join '+ sale_count_list[l]+' b on a.borough_id=b.borough and a.nbhd=b.nbhd \
  order by borough, transaction_volume desc, property_count desc, average_sale_price desc'
    mycursor.execute(sql_text3)

In [80]:
import csv
for ind in exp_list:
    csv_address=ind +'.csv'
    sql_txt='select * from '+ ind
    c=mycursor.execute(sql_txt)
    rows=c.fetchall()
    csvWriter.writerows(rows)
    f = open(csv_address, 'w', newline="")
    writer = csv.writer(f)
    title=['borough_id', 'nbhd', 'name', 'transaction_volume', 'property_count', 'average_sale_price']  # get first item to get keys
    writer.writerow(title)  # keys=title you're looking for
# write the rest
    writer.writerows(rows)

    f.close()
